In [1]:
using DataFrames, LinearAlgebra, Distributions, Revise, Serialization

In [2]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

In [3]:
# TODO: add σ to every call of compute_deltas

---
#### Mean utilities

In [4]:
df = deserialize("jls/df.jls");
firm_IDs_long = df.j;
tract_IDs_long = df.t;
X = Matrix{Float64}(df[!, [:x1, :x2]])
D = Matrix{Float64}(df[!, [:d, :d2]])
Q = df.q
M = df.M;
nJ = length(unique(firm_IDs_long));
nI = 100;

In [5]:
σ = ones(2)
pars = m.set_Pars(K = 2, nI = nI, δs=ones(nJ,1));
serialize("jls/pars.jls", pars)

---

In [6]:
ec = m.make_Economy(
    firm_IDs_long,
    tract_IDs_long,
    X,
    D,
    Q,
    M,
    nI
    );


Economy with 394 firms and 3110 tracts.


In [7]:
serialize("jls/ec.jls", ec)

---

In [8]:
Threads.nthreads()

16

In [9]:
σ = ones(2);

In [10]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
deltas, q_iter = m.compute_deltas(ec, pars, σ,max_iter=1000, verbose=true);
# serialize("jls/deltas.jls", deltas);

dist = 9.673677441313089e-7, iterations = 276


In [11]:
ec.tracts[1].shares

2×1 Matrix{Float64}:
 0.01921333161438792
 0.025686558383316523

In [11]:
using BenchmarkTools

In [19]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 1
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

  0.019207 seconds (78.05 k allocations: 37.258 MiB)


In [20]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 10
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

  0.067829 seconds (414.84 k allocations: 86.107 MiB)


In [21]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 1000
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false);

  1.410458 seconds (10.33 M allocations: 1.489 GiB, 30.32% gc time)


In [24]:
Threads.nthreads()

16

In [15]:
# # this was the line that the memory profiler said was creating the vast majority of allocations
# # ...which doesn't make sense because it's not even in the inner loop
# # ...and it's the inner loop that's driving the allocations and times (according to benchmarks)
# @time for t in ec.tracts
#     t.abδ .= t.D * pars.nlcoefs
# end

___
continued in `gmm.ipynb`